# Parsing Output

## Importing libraries and establishing instances

In [ ]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
import os
from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI()
output_parser = CommaSeparatedListOutputParser()

output_parser.get_format_instructions()

## Testing the output parser

In [ ]:
#The output parser is a two stage process: 
# it gives instructions to the LLM (knowing that the LLM will output just to a string), and then converts the string into the requested format.
reply = "red, blue, green"

output_parser.parse(reply)

## Establishing the human template

In [ ]:
human_template = "{request}\n{format_instructions}"
# the line break between the request and the format instructions makes it a little easier on the LLM
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

## Establishing the chat prompt template

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])
model_request =  chat_prompt.format_prompt(request="list five mobile suits in UC Gundam",
                          format_instructions = output_parser.get_format_instructions()).to_messages()

## Request result

In [ ]:
result = model(model_request)

In [ ]:
# stage 2 of the output parser's function is to turn the string into a Python list
output_parser.parse(result.content)

## Using the Date Time output parser

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
output_parser = DatetimeOutputParser()

In [ ]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)



In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [ ]:
model_request= chat_prompt.format_prompt(request="What date was Shinzo Abe born?", format_instructions=output_parser.get_format_instructions()).to_messages()

In [ ]:
result = model(model_request, temperature=0)

In [ ]:
result.content

In [ ]:
output_parser.parse(result.content)

## Using the output fixing parser to fix the output in the case of extraneous information

In [ ]:
from langchain.output_parsers import OutputFixingParser
misformatted = result.content

In [ ]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)
# This runs the output of the parser through the LLM one more time, hopefully to correct any mistakes in the output format. It can also be a system instruction.
new_parser.parse(misformatted)

## Using the Pydantic output parser

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
# At the moment there's no Langchain library to parse to a normal Python object, so we're using Pydantic

## Create an instance of a Pydantic class (note to self: learn more about Pydantic)

In [ ]:
class Scientist(BaseModel):
    name: str = Field(description= "Name of a Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [ ]:
parser = PydanticOutputParser(pydantic_object = Scientist)

In [ ]:
print(parser.get_format_instructions())
# The input to this output parser needs to be in a very specific JSON format

In [ ]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [ ]:
request = chat_prompt.format_prompt(request = "Tell me about a famous scientist",
                                    format_instructions=parser.get_format_instructions()).to_messages()

In [ ]:
result = model(request, temperature = 0)
# a lower temp makes it more likely to follow the format

In [ ]:
parser.parse(result.content)

In [ ]:
type(parser.parse(result.content))
# checking that it is indeed an object